# EZ - Draw Code

##### BME 271 Final Project - Fall '22 - Dr. Bucholz

Video: https://www.youtube.com/watch?v=NcKfeD3WEDk&feature=youtu.be

Git Repo: https://github.com/pradneshkolluru/bme271_proj

Project is dedicated to all the students, both old and young, struggling in drawing class!

By: Ashwin Gadiraju, Sarah Goddard, Pradnesh Kolluru

## Home Directory

In [1]:
pwd

'/Users/pradneshkolluru/Library/CloudStorage/OneDrive-DukeUniversity/Courses/BME271/bme271_proj'

In [2]:
ls #directory structure

Drawings/    Processed/   empty.jpg    main.ipynb*  tracker.csv  worksheets/


## Load Libraries


In [3]:
import glob
import os
import numpy as np
import cv2 as cv
import cv2 as cv2
from matplotlib import pyplot as plt
from IPython.display import Image as Image
from IPython.display import display
from IPython.display import clear_output
import PIL #image processing
import ipywidgets as widgets #displaying intermediary steps
import pandas as pd
from fpdf import FPDF #generate pdfs

from IPython.display import IFrame  #displaying intermediary steps


## Registering and Tracking Files

Keep track of raw input files and processed intermediary image file paths within a pandas dataframe and dictionary

In [4]:
def initiliazeTrackers():
    
    '''
    Create a data frame and dictionary to track files. For Index rown, read names from raw input files
    
    '''
    
    dictTrack = {}

    for filepath in glob.glob('./Drawings/*.jpeg'):

        name = filepath.split("/")[-1].split(".")[0]

        dictTrack[name] = {'raw': filepath, '1': '', '2': '', '3': '', '4': '', '5': ''}


    frameTrack = pd.DataFrame.from_dict(dictTrack, orient='index')
    
    return (dictTrack, frameTrack)



In [5]:
fileTrack, trackDf = initiliazeTrackers()

In [6]:
fileTrack

{'zebra': {'raw': './Drawings/zebra.jpeg',
  '1': '',
  '2': '',
  '3': '',
  '4': '',
  '5': ''},
 'squirrel': {'raw': './Drawings/squirrel.jpeg',
  '1': '',
  '2': '',
  '3': '',
  '4': '',
  '5': ''},
 'bird': {'raw': './Drawings/bird.jpeg',
  '1': '',
  '2': '',
  '3': '',
  '4': '',
  '5': ''},
 'elephant': {'raw': './Drawings/elephant.jpeg',
  '1': '',
  '2': '',
  '3': '',
  '4': '',
  '5': ''},
 'eagle': {'raw': './Drawings/eagle.jpeg',
  '1': '',
  '2': '',
  '3': '',
  '4': '',
  '5': ''}}

In [7]:
trackDf

,raw,1,2,3,4,5
zebra,./Drawings/zebra.jpeg,,,,,
squirrel,./Drawings/squirrel.jpeg,,,,,
bird,./Drawings/bird.jpeg,,,,,
elephant,./Drawings/elephant.jpeg,,,,,
eagle,./Drawings/eagle.jpeg,,,,,


In [8]:

def updateTracker(repo, frame):
    
    '''
    Every time a new image is created or processed, the tracker dataframes and dictionary
    will be updated with assosciated filepaths. Initially populates tracker structures
    when starting program
    '''

    for name in repo:
        
        print(name)

        processedPath = os.path.join("./Processed", name)

        if not os.path.exists(processedPath):
            os.mkdir(processedPath)
            #print("Directory '%s' created" %processedPath)
            
        else:
            
            for filepath in glob.glob(processedPath + '/*.jpg'):
                
                num = filepath.split("/")[-1].split('.')[0]
                repo[name][num] = filepath
                
                frame.loc[name][num] = filepath
                
    frame.to_csv("./tracker.csv")

In [9]:
updateTracker(fileTrack, trackDf)

zebra
squirrel
bird
elephant
eagle


In [10]:
fileTrack

{'zebra': {'raw': './Drawings/zebra.jpeg',
  '1': './Processed/zebra/1.jpg',
  '2': './Processed/zebra/2.jpg',
  '3': './Processed/zebra/3.jpg',
  '4': './Processed/zebra/4.jpg',
  '5': './Processed/zebra/5.jpg'},
 'squirrel': {'raw': './Drawings/squirrel.jpeg',
  '1': './Processed/squirrel/1.jpg',
  '2': './Processed/squirrel/2.jpg',
  '3': './Processed/squirrel/3.jpg',
  '4': './Processed/squirrel/4.jpg',
  '5': './Processed/squirrel/5.jpg'},
 'bird': {'raw': './Drawings/bird.jpeg',
  '1': './Processed/bird/1.jpg',
  '2': './Processed/bird/2.jpg',
  '3': './Processed/bird/3.jpg',
  '4': './Processed/bird/4.jpg',
  '5': './Processed/bird/5.jpg'},
 'elephant': {'raw': './Drawings/elephant.jpeg',
  '1': './Processed/elephant/1.jpg',
  '2': './Processed/elephant/2.jpg',
  '3': './Processed/elephant/3.jpg',
  '4': './Processed/elephant/4.jpg',
  '5': './Processed/elephant/5.jpg'},
 'eagle': {'raw': './Drawings/eagle.jpeg',
  '1': './Processed/eagle/1.jpg',
  '2': './Processed/eagle/2.jpg'

In [11]:
trackDf

,raw,1,2,3,4,5
zebra,./Drawings/zebra.jpeg,./Processed/zebra/1.jpg,./Processed/zebra/2.jpg,./Processed/zebra/3.jpg,./Processed/zebra/4.jpg,./Processed/zebra/5.jpg
squirrel,./Drawings/squirrel.jpeg,./Processed/squirrel/1.jpg,./Processed/squirrel/2.jpg,./Processed/squirrel/3.jpg,./Processed/squirrel/4.jpg,./Processed/squirrel/5.jpg
bird,./Drawings/bird.jpeg,./Processed/bird/1.jpg,./Processed/bird/2.jpg,./Processed/bird/3.jpg,./Processed/bird/4.jpg,./Processed/bird/5.jpg
elephant,./Drawings/elephant.jpeg,./Processed/elephant/1.jpg,./Processed/elephant/2.jpg,./Processed/elephant/3.jpg,./Processed/elephant/4.jpg,./Processed/elephant/5.jpg
eagle,./Drawings/eagle.jpeg,./Processed/eagle/1.jpg,./Processed/eagle/2.jpg,./Processed/eagle/3.jpg,./Processed/eagle/4.jpg,./Processed/eagle/5.jpg


# Generate Intermediary Drawing Images

In [12]:
def displayImArray(rawPath, path1 = None, path2 = None, path3 = None, path4 = None, path5 = None):
    
    '''
    Displays raw and generated process images with in python notebook as widgets
    
    '''
    
    raw = open(rawPath,'rb').read()
    
    img1 = open("./empty.jpg",'rb').read()
    img2 = open("./empty.jpg",'rb').read()
    img3 = open("./empty.jpg",'rb').read()
    img4 = open("./empty.jpg",'rb').read()
    img5 = open("./empty.jpg",'rb').read()
    
    
    if path1:
        
        img1 = open(path1,'rb').read()
        
        if path2:
            
            img2 = open(path2,'rb').read()
            
            if path3:
            
                img3 = open(path3,'rb').read()
                
                if path4:
            
                    img4 = open(path4,'rb').read()
                
                    if path5:
            
                        img5 = open(path5,'rb').read()
    
    #Display Dimensions
    w = 300
    h = 400
    
    wi0 = widgets.Image(value=raw, format='jpg', width=w, height=h)
    wi1 = widgets.Image(value=img1, format='jpg', width=w, height=h)
    wi2 = widgets.Image(value=img2, format='jpg', width=w, height=h)
    wi3 = widgets.Image(value=img3, format='jpg', width=w, height=h)
    wi4 = widgets.Image(value=img4, format='jpg', width=w, height=h)
    wi5 = widgets.Image(value=img5, format='jpg', width=w, height=h)
    
    
    a=[wi0, wi1, wi2, wi3, wi4, wi5]
    wid=widgets.HBox(a)
    display(wid)
    
    
def saveProcess(name, num, arr):
    
    '''
    Save processed files in correct directory according to raw name, and process num
    '''
    
    im = PIL.Image.fromarray(arr)
    
    savePath = os.path.join("./Processed", name, num + ".jpg")
    
    im.save(savePath)
    
    return(savePath)

            
def findProp(num, prop):
    
    '''
    Custom multiplacation function that completes multi operation and returns closes odd int
    '''
    
    div = num // (1 / prop)
    
    div = int(div)
    
    if div % 2 == 0:
        
        return div + 1
    
    else:
        return div
    
    
def getComplexVal(idx):
    
    '''
    Translates user input complexity val into median blur value
    '''
    
    hardVals = np.linspace(1, 10, 10)
    medVals = np.arange(21, 41, 2)
    
    medVals = medVals[::-1]
    
    verbose = False
    
    if verbose:
        print(hardVals)
        print(medVals)
        print(len(hardVals) == len(medVals))
        
    return(int(medVals[idx - 1]))
    

In [13]:

def makeDrawFiles(name, dictTrack, frameTrack):
    
    '''
    Guide user through processing pipeline, asking for input such as what raw input, complexity, and satisfaction.
    Loads and saves processed files into correct directories.
    '''
    
    rawPath = dictTrack[name]['raw']
    
    raw = cv.imread(rawPath,0)
    img = cv.imread(rawPath,0)

    
    
    while True:
        
        
        displayImArray(rawPath)
        
        diff = 0
        
        while True:
            
            diff = int(input('How complicated do you want your starter image to be? (From 1 - 10). Easy to Hard    '))
            
            if diff >= 1 and diff <= 10:     
                break
                
        medVal = getComplexVal(diff)
                
        ###### Process 1
        img_blur = cv2.medianBlur(img, medVal)
        edges1 = cv.Canny(img_blur,1,100)
        
        p1 = 255 - edges1
        
        onePath = saveProcess(name, "1", p1)
        
        displayImArray(rawPath, onePath)
        
        
        ###### Step 1 Validation
        
        satisfied = input("Are you satisfied? (yes or no)")
        
        if (satisfied.lower() == 'no'):
            
            clear_output()
            
            print("Try another complexity value!")
            
            continue
        
        
        ###### Process 2
        
        img_blur2 = cv2.medianBlur(img, findProp(medVal, .6))
        
        edges2 = cv.Canny(img_blur2,1,100)
        
        p2 = 255 - edges2
        
        twoPath = saveProcess(name, "2", p2)
        
        #displayImArray(rawPath, onePath, twoPath)
        
        
        ###### Process 3
        
        img_blur3 = cv2.medianBlur(img, findProp(medVal, .355))
        edges3 = cv.Canny(img_blur3,1,100)
        
        p3 = 255 - edges3
        
        threePath = saveProcess(name, "3", p3)
        
        #displayImArray(rawPath, onePath, twoPath)
        
        
        ###### Process 4
        
        img_blur4 = cv2.medianBlur(img, findProp(medVal, 2.29))
        
        img_blur4_2 = cv2.medianBlur(img, findProp(medVal, .355))
        edges4 = cv.Canny(img_blur4_2, 1, 100)
        edges4 = edges4 * 70
        
        p4 = (255 - edges4) + img_blur4
        
        fourPath = saveProcess(name, "4", p4)
        
        
        ###### Process 5
        
        img_blur5 = cv2.medianBlur(img, medVal)
        
        img_blur5_2 = cv2.medianBlur(img, findProp(medVal, .355))
        edges5 = cv.Canny(img_blur5_2, 1, 100)
        edges5 = edges5 * 70
        
        p5 = (255 - edges5) + img_blur5
        
        fivePath = saveProcess(name, "5", p5)
        
        
        #Display
        displayImArray(rawPath, onePath, twoPath, threePath, fourPath, fivePath)
        
    
        break
        

    
    
    
    

In [14]:
makeDrawFiles("eagle", fileTrack, trackDf)

How complicated do you want your starter image to be? (From 1 - 10). Easy to Hard    6


Are you satisfied? (yes or no)yes


In [15]:
trackDf

,raw,1,2,3,4,5
zebra,./Drawings/zebra.jpeg,./Processed/zebra/1.jpg,./Processed/zebra/2.jpg,./Processed/zebra/3.jpg,./Processed/zebra/4.jpg,./Processed/zebra/5.jpg
squirrel,./Drawings/squirrel.jpeg,./Processed/squirrel/1.jpg,./Processed/squirrel/2.jpg,./Processed/squirrel/3.jpg,./Processed/squirrel/4.jpg,./Processed/squirrel/5.jpg
bird,./Drawings/bird.jpeg,./Processed/bird/1.jpg,./Processed/bird/2.jpg,./Processed/bird/3.jpg,./Processed/bird/4.jpg,./Processed/bird/5.jpg
elephant,./Drawings/elephant.jpeg,./Processed/elephant/1.jpg,./Processed/elephant/2.jpg,./Processed/elephant/3.jpg,./Processed/elephant/4.jpg,./Processed/elephant/5.jpg
eagle,./Drawings/eagle.jpeg,./Processed/eagle/1.jpg,./Processed/eagle/2.jpg,./Processed/eagle/3.jpg,./Processed/eagle/4.jpg,./Processed/eagle/5.jpg


In [16]:
fileTrack

{'zebra': {'raw': './Drawings/zebra.jpeg',
  '1': './Processed/zebra/1.jpg',
  '2': './Processed/zebra/2.jpg',
  '3': './Processed/zebra/3.jpg',
  '4': './Processed/zebra/4.jpg',
  '5': './Processed/zebra/5.jpg'},
 'squirrel': {'raw': './Drawings/squirrel.jpeg',
  '1': './Processed/squirrel/1.jpg',
  '2': './Processed/squirrel/2.jpg',
  '3': './Processed/squirrel/3.jpg',
  '4': './Processed/squirrel/4.jpg',
  '5': './Processed/squirrel/5.jpg'},
 'bird': {'raw': './Drawings/bird.jpeg',
  '1': './Processed/bird/1.jpg',
  '2': './Processed/bird/2.jpg',
  '3': './Processed/bird/3.jpg',
  '4': './Processed/bird/4.jpg',
  '5': './Processed/bird/5.jpg'},
 'elephant': {'raw': './Drawings/elephant.jpeg',
  '1': './Processed/elephant/1.jpg',
  '2': './Processed/elephant/2.jpg',
  '3': './Processed/elephant/3.jpg',
  '4': './Processed/elephant/4.jpg',
  '5': './Processed/elephant/5.jpg'},
 'eagle': {'raw': './Drawings/eagle.jpeg',
  '1': './Processed/eagle/1.jpg',
  '2': './Processed/eagle/2.jpg'

## Create PDF

In [17]:
def genPdf(name, df):
    
    '''
    From animal name, create a pdf worksheet displaying step by step instructions to draw animal
    '''
    
    
    if df.loc[name].isnull().any():
        
        raise Exception("Need 5 Processed Drawing Files")

    pdf = FPDF() # Initializing object
    pdf.add_page() # Adding new page


    pdf.set_fill_color(200,0,0)
    pdf.set_text_color(255,255,255)
    pdf.set_font("Arial", size=30, style='B')
    pdf.set_xy(10,32)
    pdf.cell(0, 12.5, txt= (name + ' Drawing Instructions').upper(), ln=1, align="C" , fill = True)


    w = 50
    h = 50

    start_y = 110
    space = 15

    pdf.image(df.loc[name]['raw'], x= 15, y= start_y, w=w, h=h)
    pdf.image(df.loc[name]['1'], x= space * 2 + w, y= start_y, w=w, h=h)
    pdf.image(df.loc[name]['2'], x= space * 3 + 2 * w, y= start_y, w=w, h=h)

    pdf.image(df.loc[name]['3'], x=15, y= 2 * space + h + start_y, w=w, h=h)
    pdf.image(df.loc[name]['4'], x= space * 2 + w, y= 2 * space + h + start_y, w=w, h=h)
    pdf.image(df.loc[name]['5'], x= space * 3 + 2 * w, y= 2 * space + h + start_y, w=w, h=h)
    
    
    outpath = "./worksheets/" + name + ".pdf"
    pdf.output(outpath)

    
    display(IFrame(outpath, width=200, height=200))
    
    
    

In [18]:
genPdf("eagle", trackDf)

## Putting it Together - Run Complete Pipeline

In [19]:
def comPipe():
    
    
    repo, frame = initiliazeTrackers()
    
    updateTracker(repo, frame)
    
    while True:
        
        name = input("What Animal? ")
        name = name.lower()
        
        if name not in frame.index:
            print("****" + name + " not in drawing repo, please select from the options above." + "****")
        
        else:
            break
    
    
    makeDrawFiles(name, repo, frame)
    genPdf(name, frame)
    

In [20]:
comPipe()

zebra
squirrel
bird
elephant
eagle
What Animal? alligator
****alligator not in drawing repo, please select from the options above.****
What Animal? elephant


How complicated do you want your starter image to be? (From 1 - 10). Easy to Hard    7


Are you satisfied? (yes or no)yes


## Copyright Statement
All Rights Reserved.

All material appearing in the E-Z Draw code repo is protected by copyright under U.S. Copyright laws and is the property of BME 271 Project Group or the party credited as the provider of the content. You may not copy, reproduce, distribute, publish, display, perform, modify, create derivative works, transmit, or in any way exploit any such content, nor may you distribute any part of this content over any network, including a local area network, sell or offer it for sale, or use such content to construct any kind of database. You may not alter or remove any copyright or other notice from copies of the content in E-Z Draw code repo. Copying or storing any content except as provided above is expressly prohibited without prior written permission of the University or the copyright holder identified in the individual content’s copyright notice.